In [1]:
import pickle
from pytorch3d.structures import Meshes
import torch
import numpy as np
import math


import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
# os.environ['NCCL_P2P_DISABLE'] = '1'
# os.environ['NCCL_IB_DISABLE'] = '1'
# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# from training.volumetric_rendering.camera_utils2 import transform_to_view_space, project_to_clip_space

from collections import OrderedDict
from pytorch3d.io import load_obj, save_obj

In [2]:
_v_template, faces, aux = load_obj('assets/flame_uv_no_back_close_mouth_no_subdivision.obj', load_textures=False)

_faces = faces.verts_idx
_faces_uvs =  faces.textures_idx
_vertex_uvs = aux.verts_uvs

An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/home/rabdal/anaconda3/envs/pytorch3d/lib/python3.10/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/home/rabdal/anaconda3/envs/pytorch3d/lib/python3.10/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt


In [3]:
def calculate_triangle_area(vertices):
    # Calculate the cross product of two edges of the triangle
    edge1 = vertices[1] - vertices[0]
    edge2 = vertices[2] - vertices[0]
    cross_product = np.cross(edge1, edge2)
    
    # Calculate the area as half the magnitude of the cross product
    area = 0.5 * np.linalg.norm(cross_product)
    return area

def generate_random_point_in_tetrahedron(vertices):
    # Generate random barycentric coordinates
    r1 = np.random.random()
    r2 = np.random.random()
    r3 = np.random.random()
    r4 = np.random.random()

    # print(r1, r2, r3)

    # Ensure that the sum of barycentric coordinates is <= 1
    total = r1 + r2 + r3 + r4
    r1 /= total
    r2 /= total
    r3 /= total
    r4 /= total

    # Calculate the fourth barycentric coordinate
    # r4 = 1.0 - r1 - r2 - r3

    # print(r1,r2,r3,r4, r1+r2+r3+r4)

    # Calculate the random point using barycentric coordinates
    point = r1 * vertices[0] + r2 * vertices[1] + r3 * vertices[2] + r4 * vertices[3]

    return point, [r1, r2, r3, r4]


class points:
    def __init__(self, coordinates= [], upper_traingle= [], lower_traingle= [], upper_uv= [], lower_uv= [], tet_bari= [], tex_up= [], tex_low= [], 
                 z_0 = [], z = [], z_1 = [], tet_faces = [], faces = []):
        self.coordinates = coordinates
        self.upper_traingle = upper_traingle
        self.lower_traingle = lower_traingle
        self.upper_uv = upper_uv
        self.lower_uv = lower_uv
        self.tet_bari = tet_bari
        self.tex_up = tex_up
        self.tex_low = tex_low
        self.z_0 = z_0
        self.z = z
        self.z_1 = z_1
        self.tet_faces = tet_faces
        self.faces = faces



@torch.no_grad()
def generate_points(num_shells, faces, vertices, tex_coord, uv_faces):
   
        offset = torch.max(faces.reshape(-1)) 
        offset2 = torch.max(uv_faces.reshape(-1))

        orig_vertices = vertices.to(faces.device).reshape(-1, 3)
        vertices = orig_vertices
        tex_coord = tex_coord.to(faces.device).reshape(-1, 2).repeat(2,1)

        new_faces = []

        face_one = faces
        uv_faceone = uv_faces

        point_list = points()
        kk = 0 
        for i in range(1, num_shells):
            
            face_two = face_one + offset + 1
            uv_facetwo = uv_faceone + offset2 + 1
            for face1, face2, uv_face1, uv_face2 in zip(face_one, face_two, uv_faceone, uv_facetwo):
             

                if i == 1:
                    new_faces.append(face1.to(faces.device))

                tetrahedra = []
                # print()
                tetrahedra.append([vertices[face1[0]], vertices[face1[1]], vertices[face1[2]], vertices[face2[0]]])
                tetrahedra.append([vertices[face2[0]],vertices[face2[1]], vertices[face2[2]], vertices[face1[1]]])
                tetrahedra.append([vertices[face1[1]], vertices[face1[2]], vertices[face2[2]], vertices[face2[0]]])

                tetrahedra_faces = []
                tetrahedra_faces.append(torch.tensor([uv_face1[0], uv_face1[1], uv_face1[2], uv_face2[0]]))
                tetrahedra_faces.append(torch.tensor([uv_face2[0], uv_face2[1], uv_face2[2], uv_face1[1]]))
                tetrahedra_faces.append(torch.tensor([uv_face1[1], uv_face1[2], uv_face2[2], uv_face2[0]]))
                
                tetrahedra_tri_faces = []

                tetrahedra_tri_faces.append(torch.tensor([face1[0], face1[1], face1[2], face2[0]]))
                tetrahedra_tri_faces.append(torch.tensor([face2[0], face2[1], face2[2], face1[1]]))
                tetrahedra_tri_faces.append(torch.tensor([face1[1], face1[2], face2[2], face2[0]]))

                weight_ = 1

                for ii in range(3):
                    for jj in range(weight_):
                
                   
                        point, list_of_bari_tets = generate_random_point_in_tetrahedron(tetrahedra[ii])
                        tet_faces = tetrahedra_faces[ii]
                        tri_faces = tetrahedra_tri_faces[ii]
                     

                        point_list.coordinates.append(point)
                        point_list.tet_bari.append(torch.tensor(list_of_bari_tets))
                        point_list.tet_faces.append(tet_faces)
                        point_list.faces.append(tri_faces)   
                     
                        kk+=2

                
                new_faces.append(face1.to(faces.device))

                new_faces.append(face2.to(faces.device))

            face_one = face_two
            uv_faceone = uv_facetwo
        
        # new_faces = torch.stack(new_faces)
        point_list.coordinates = torch.stack(point_list.coordinates)
        point_list.tet_bari = torch.stack(point_list.tet_bari)
        point_list.tet_faces = torch.stack(point_list.tet_faces)
        point_list.faces = torch.stack(point_list.faces)
       
        return vertices, point_list




@torch.no_grad()
def settings_(vertices, faces, uv_faces, num_shells, tex_coord, normal_scale=0.1, disable_small = False):
                faces = torch.tensor(faces.astype(np.int64)).to(vertices.device)
                print(faces[None].repeat(vertices.shape[0], 1, 1).shape)
                print('NORMAL EXTENSION')
                mesh = Meshes(
                    verts=vertices, faces=faces[None].repeat(vertices.shape[0], 1, 1)
                )
                orig_vertex_ = vertices.clone()
                # bs, n, 3
                vertex_normal = mesh.verts_normals_padded()

                linscale = torch.linspace(normal_scale, -normal_scale, num_shells)

                # kfs
                if disable_small:  # disable small meshes (meshes that are inside SMPL)
                    linscale = torch.linspace(0, 0, num_shells)
                linscale = linscale.to(vertices.device).to(vertices.dtype)
                vertices = (
                    vertices[:, None]
                    + linscale.reshape(1, num_shells, 1, 1) * vertex_normal[:, None]
                )
                
                vertices, points_list = generate_points(num_shells, faces, vertices, tex_coord, uv_faces)                

                return vertices, points_list

num_shells = 5
vertices__, points_list = settings_((_v_template*5).unsqueeze(0), _faces.numpy(), _faces_uvs, num_shells, _vertex_uvs, normal_scale=0.04, disable_small = False)
texc = _vertex_uvs.clone().detach().unsqueeze(0).repeat(num_shells,1,1).float()

torch.Size([1, 42024, 3])
NORMAL EXTENSION
1
2
3
4


In [4]:

print('Points:', points_list.coordinates.shape)
print('Faces_per_point:', points_list.faces.shape)
print('uvw_per_point', points_list.tet_bari.shape)
print('vertices_flattened', vertices__.shape)


# Given the face indices, find the vertices of the triangles
def indexed_tensor(faces, vertices):
    # Extract the indices for each vertex in a face
    indices = faces.view(-1)
    
    # Use torch.index_select to get the indexed tensor
    indexed_tensor = torch.index_select(vertices, dim=0, index=indices)
    
    # Reshape the indexed tensor to match the original faces shape
    indexed_tensor = indexed_tensor.view(faces.shape[0], faces.shape[1], -1)

    return indexed_tensor

# Find the points using baricentric interpolation
def combine_vertices(indexed_tensor, weights):
    # Make sure the shapes are compatible
    assert indexed_tensor.shape[0] == weights.shape[0]
    
    # Expand the weights tensor to have the same shape as indexed_tensor
    expanded_weights = weights.unsqueeze(2)
    
    # Multiply the indexed tensor by the expanded weights and sum along the second dimension
    combined_tensor = torch.sum(indexed_tensor * expanded_weights, dim=1)
    
    return combined_tensor

# This is where the deformation takes place (per vertex)
vertices__ = vertices__  + torch.randn(vertices__.shape)

#These are the new triangles where the faces are the same by the vertex values are different
indexed_tensor_ = indexed_tensor(points_list.faces, vertices__)  # bs x f x 3 x 3

# Do the interpolation based on the saved uvw to generate dispalcement in the points
new_coordinates  = combine_vertices(indexed_tensor_, points_list.tet_bari)

# The points are different from original points (if the deformation is not added then points should be the same)
print(torch.unique(points_list.coordinates - new_coordinates))

Points: torch.Size([504288, 3])
Faces_per_point: torch.Size([504288, 4])
uvw_per_point torch.Size([504288, 4])
vertices_flattened torch.Size([105570, 3])
tensor([0.])


In [ ]:
# Creating the gaussian model 
gaussians = GaussianModel(points_list, texc ,3)